# Loading the dollar-to-ruble exchange rate into the SQL database

## Creating an SQL database

In [1]:
import bs4
import requests
import pandas as pd
import datetime
from sqlalchemy import create_engine

# Parsing the dollar rate from 01.01.2000 to 15.06.2020 
# from the site cbr.ru
html = 'https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ=R01235&UniDbQuery.From=01.01.2020&UniDbQuery.To=15.06.2020'

req = requests.get(html)
parser = bs4.BeautifulSoup(req.text, "lxml")

rows = parser.find_all("tr")[2:]

ex_rate = []
for row in rows:
    cells = row.find_all('td')
    ex_rate.append(cells)

for i in range(len(ex_rate)):
    ex_rate[i][0] = ex_rate[i][0].text
    ex_rate[i][1] = ex_rate[i][1].text
    ex_rate[i][2] = ex_rate[i][2].text.replace(',','.')

ex_rate = pd.DataFrame(ex_rate)
del ex_rate[1]
ex_rate.columns = ['date', 'exchange_rate']
ex_rate['date'] = pd.to_datetime(ex_rate['date'], format='%d.%m.%Y')
ex_rate['exchange_rate'] = pd.to_numeric(ex_rate['exchange_rate'])

In [2]:
ex_rate.head()

,date,exchange_rate
0,2020-01-01,61.9057
1,2020-01-10,61.2340
2,2020-01-11,61.2632
3,2020-01-14,60.9474
4,2020-01-15,61.4140


In [3]:
ex_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
date             101 non-null datetime64[ns]
exchange_rate    101 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.7 KB


In [4]:
# Loading data into the SQL database

engine = create_engine('sqlite:///exchange_rate.db')
ex_rate.to_sql('exchange_rate', con=engine, if_exists = 'replace', index=False)

In [5]:
# Dollar to ruble exchange rate in SQL

pd.read_sql_query('SELECT * FROM exchange_rate', con=engine)

,date,exchange_rate
0,2020-01-01 00:00:00.000000,61.9057
1,2020-01-10 00:00:00.000000,61.2340
2,2020-01-11 00:00:00.000000,61.2632
3,2020-01-14 00:00:00.000000,60.9474
4,2020-01-15 00:00:00.000000,61.4140
5,2020-01-16 00:00:00.000000,61.4328
6,2020-01-17 00:00:00.000000,61.5694
7,2020-01-18 00:00:00.000000,61.5333
8,2020-01-21 00:00:00.000000,61.4654
9,2020-01-22 00:00:00.000000,61.8552


## Loading the current course into the SQL database

In [6]:
import bs4
import requests
import pandas as pd
import datetime
from sqlalchemy import create_engine

# Parsing the dollar exchange rate on the current date 
# from the site cbr.ru

now_date = datetime.datetime.now().strftime('%d.%m.%Y')

str1 = 'https://www.cbr.ru/currency_base/dynamics/?UniDbQuery.Posted=True&UniDbQuery.mode=1&UniDbQuery.date_req1=&UniDbQuery.date_req2=&UniDbQuery.VAL_NM_RQ=R01235&UniDbQuery.From='
str2 = '&UniDbQuery.To='

html = str1+now_date+str2+now_date

req = requests.get(html)
parser = bs4.BeautifulSoup(req.text, "lxml")

rows = parser.find_all("tr")[2:]

ex_rate = []
for row in rows:
    cells = row.find_all('td')
    ex_rate.append(cells)

for i in range(len(ex_rate)):
    ex_rate[i][0] = ex_rate[i][0].text
    ex_rate[i][1] = ex_rate[i][1].text
    ex_rate[i][2] = ex_rate[i][2].text.replace(',','.')

ex_rate = pd.DataFrame(ex_rate)
del ex_rate[1]
ex_rate.columns = ['date', 'exchange_rate']
ex_rate['date'] = pd.to_datetime(ex_rate['date'], format='%d.%m.%Y')
ex_rate['exchange_rate'] = pd.to_numeric(ex_rate['exchange_rate'])

In [7]:
ex_rate.head()

,date,exchange_rate
0,2020-06-17,69.7524


In [8]:
ex_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
date             1 non-null datetime64[ns]
exchange_rate    1 non-null float64
dtypes: datetime64[ns](1), float64(1)
memory usage: 96.0 bytes


In [9]:
# Loading new data into the SQL database

engine = create_engine('sqlite:///exchange_rate.db')
ex_rate.to_sql('exchange_rate', con=engine, if_exists = 'append', index=False)

In [10]:
# Dollar to ruble exchange rate in SQL

pd.read_sql_query('SELECT * FROM exchange_rate', con=engine)

,date,exchange_rate
0,2020-01-01 00:00:00.000000,61.9057
1,2020-01-10 00:00:00.000000,61.2340
2,2020-01-11 00:00:00.000000,61.2632
3,2020-01-14 00:00:00.000000,60.9474
4,2020-01-15 00:00:00.000000,61.4140
5,2020-01-16 00:00:00.000000,61.4328
6,2020-01-17 00:00:00.000000,61.5694
7,2020-01-18 00:00:00.000000,61.5333
8,2020-01-21 00:00:00.000000,61.4654
9,2020-01-22 00:00:00.000000,61.8552
